In [1]:
import pyemu
import os, shutil
import pandas as pd
import flopy
import numpy as np
import platform

flopy is installed in /Users/jwhite/Dev/flopy/flopy


# This notebook is designed to show high-level PST file generation and manipulation examples, and also highlight some of the steps that `Schur` object functions perform in a bit more detail

The assumption is that a user has a model that `flopy` can grok. That's all we need, using the BOSS PEST-ification techniques `pyemu` has. 

In [2]:
nam_file = "freyberg.nam"
org_model_ws = "Freyberg_transient"
new_model_ws = "pest_setup"

# load the model, change dir and run once to get a hydmod output file and list file
m = flopy.modflow.Modflow.load(nam_file,model_ws=org_model_ws,check=False)
m.change_model_ws("temp",reset_external=True)
m.name = nam_file.split(".")[0]

# let's just retain the calibration data for now by trimming HYDMOD
hyd = m.get_package('hyd')
hyddf = pd.DataFrame(hyd.obsdata)
#hyddf = hyddf.loc[[True if 'cr' in i.decode() else False for i in hyddf.hydlbl]]
hyd.obsdata = hyddf.to_records(index=False).astype(hyd.obsdata.dtype)
hyd.nhyd = len(hyd.obsdata)
m.exe_name = 'mfnwt'
m.write_input()

EXE_DIR = os.path.join("..","bin")
if "window" in platform.platform().lower():
    EXE_DIR = os.path.join(EXE_DIR,"win")
elif "darwin" in platform.platform().lower():
    EXE_DIR = os.path.join(EXE_DIR,"win")
else:
    EXE_DIR = os.path.join(EXE_DIR,"linux")

[shutil.copy2(os.path.join(EXE_DIR,f),os.path.join('temp',f)) for f in os.listdir(EXE_DIR)]

pyemu.helpers.run('{0} {1}'.format(m.exe_name,nam_file), cwd='temp')


changing model workspace...
   temp


## Let's make some parameters. How about zones for HK and a constants for SY and RCH?

In [3]:
zn_array = np.loadtxt(os.path.join("Freyberg_truth","hk.zones"))
k_zone_dict = {k:zn_array for k in range(m.nlay)}
const_props = []
const_props.append(["upw.sy", None])
const_props.append(["rch.rech",None])
zone_props = [['upw.hk',0]]

## Maybe we ought to also treat well pumping as an uncertain parameter

In [4]:
bc_props = [["wel.flux",None]]

In [5]:
mfp = pyemu.helpers.PstFromFlopyModel(m,new_model_ws="schur_test",zone_props=zone_props,
                                          const_props=const_props,k_zone_dict=k_zone_dict,
                                          remove_existing=True,temporal_bc_props=bc_props)

2019-09-02 10:19:47.082093 starting: updating model attributes
2019-09-02 10:19:47.082396 finished: updating model attributes took: 0:00:00.000303
2019-09-02 10:19:47.082598 WARNING: removing existing 'new_model_ws

creating model workspace...
   schur_test

changing model workspace...
   schur_test
2019-09-02 10:19:48.192078 starting: writing new modflow input files
Util2d:delr: resetting 'how' to external
Util2d:delc: resetting 'how' to external
Util2d:model_top: resetting 'how' to external
Util2d:botm_layer_0: resetting 'how' to external
Util2d:botm_layer_1: resetting 'how' to external
Util2d:botm_layer_2: resetting 'how' to external
Util2d:ibound_layer_0: resetting 'how' to external
Util2d:ibound_layer_1: resetting 'how' to external
Util2d:ibound_layer_2: resetting 'how' to external
Util2d:strt_layer_0: resetting 'how' to external
Util2d:strt_layer_1: resetting 'how' to external
Util2d:strt_layer_2: resetting 'how' to external
Util2d:rech_1: resetting 'how' to external
Util2d:rech_

Util2d:rech_285: resetting 'how' to external
Util2d:rech_286: resetting 'how' to external
Util2d:rech_287: resetting 'how' to external
Util2d:rech_288: resetting 'how' to external
Util2d:rech_289: resetting 'how' to external
Util2d:rech_290: resetting 'how' to external
Util2d:rech_291: resetting 'how' to external
Util2d:rech_292: resetting 'how' to external
Util2d:rech_293: resetting 'how' to external
Util2d:rech_294: resetting 'how' to external
Util2d:rech_295: resetting 'how' to external
Util2d:rech_296: resetting 'how' to external
Util2d:rech_297: resetting 'how' to external
Util2d:rech_298: resetting 'how' to external
Util2d:rech_299: resetting 'how' to external
Util2d:rech_300: resetting 'how' to external
Util2d:rech_301: resetting 'how' to external
Util2d:rech_302: resetting 'how' to external
Util2d:rech_303: resetting 'how' to external
Util2d:rech_304: resetting 'how' to external
Util2d:rech_305: resetting 'how' to external
Util2d:rech_306: resetting 'how' to external
Util2d:rec

2019-09-02 10:20:09.854087 starting: running pestchek on freyberg.pst
2019-09-02 10:20:09.857662 finished: running pestchek on freyberg.pst took: 0:00:00.003575
2019-09-02 10:20:09.857868 starting: building prior covariance matrix
2019-09-02 10:20:09.884555 saving prior covariance matrix to file schur_test/freyberg.pst.prior.cov
2019-09-02 10:20:09.885571 finished: building prior covariance matrix took: 0:00:00.027703
2019-09-02 10:20:09.885808 starting: saving intermediate _setup_<> dfs into schur_test


//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


2019-09-02 10:20:10.467075 finished: saving intermediate _setup_<> dfs into schur_test took: 0:00:00.581267
2019-09-02 10:20:10.467177 all done


# Cool - let's take a quick look at the PST control file this made

In [6]:
inpst = mfp.pst

In [7]:
inpst.parameter_data

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
const_rech0__cn,const_rech0__cn,log,factor,1.0,0.10,10.0,cn_rech0,1.0,0.0,1
const_sy0__cn,const_sy0__cn,log,factor,1.0,0.10,10.0,cn_sy0,1.0,0.0,1
hk0_zone:1.0__zn,hk0_zone:1.0__zn,log,factor,1.0,0.01,100.0,zn_hk0,1.0,0.0,1
hk0_zone:2.0__zn,hk0_zone:2.0__zn,log,factor,1.0,0.01,100.0,zn_hk0,1.0,0.0,1
hk0_zone:3.0__zn,hk0_zone:3.0__zn,log,factor,1.0,0.01,100.0,zn_hk0,1.0,0.0,1
hk0_zone:4.0__zn,hk0_zone:4.0__zn,log,factor,1.0,0.01,100.0,zn_hk0,1.0,0.0,1
hk0_zone:5.0__zn,hk0_zone:5.0__zn,log,factor,1.0,0.01,100.0,zn_hk0,1.0,0.0,1
hk0_zone:6.0__zn,hk0_zone:6.0__zn,log,factor,1.0,0.01,100.0,zn_hk0,1.0,0.0,1
welflux_000,welflux_000,log,factor,1.0,0.10,10.0,welflux,1.0,0.0,1


# Looks like we have multipliers on our zones. Let's set `noptmax` to -1 to calculate a Jacobian matrix and then see what `pyemu` functionality we can light up

## First, though, we can report all the control data to see `noptmax` and everything else

In [8]:
inpst.control_data.formatted_values

name
rstfle                        restart
pestmode                   estimation
npar                                9
nobs                           240080
npargp                              4
nprior                              0
nobsgp                            641
maxcompdim                          0
ntplfle                             4
ninsfle                             3
precis                         single
dpoint                          point
numcom                              1
jacfile                             0
messfile                            0
obsreref                   noobsreref
rlambda1                 2.000000E+01
rlamfac                 -3.000000E+00
phiratsuf                3.000000E-01
phiredlam                1.000000E-02
numlam                             -7
jacupdate                         999
lamforgive                 lamforgive
derforgive               noderforgive
relparmax                1.000000E+01
facparmax                1.000000E+01
facorig

In [9]:
inpst.control_data.noptmax=-1
inpst.write(os.path.join('schur_test','freyberg_pest.pst'))

noptmax:-1, npar_adj:9, nnz_obs:240080


## Set a flag for whether to run in parallel or not

In [10]:
parallel = True

## If we just want to run in serial, we could run at the command line or we can use the following `pyemu` helper function to run `pestpp` for us. The cool thing is, this is operating system agnostic -- it will work regardless of platform.

In [11]:
if parallel==False:
    pyemu.helpers.run('pestpp freyberg_pest.pst', cwd='schur_test')

# We can also run in parallel if we want, using the `start_workers` helper. Set the number of workers you want (all we be launched locally on your machine) and reset parallel to `True`

In [12]:
if parallel == True:
    num_workers = 5
    pyemu.helpers.start_workers('schur_test','pestpp-glm','freyberg_pest.pst',num_workers=num_workers,master_dir="master_glm")

# Now, once we have a Jacobian matrix, we can create a posterior Schur complement object. If we don't pass an explicit covariance matrix, the object will use parameter bounds for the prior variance (diagonal matrix) for parameters (also note that if no `.pst` file was passed, `pyemu` will look for one with the same root name in the same location as the `.jco` file that was passed.

In [14]:
sc = pyemu.Schur(os.path.join('master_glm','freyberg_pest.jcb'),verbose=True)

In [15]:
sc.jco.x

array([[-2.15797116e+03,  0.00000000e+00,  1.15703946e-01, ...,
        -3.55650790e+02, -4.72303509e+02,  0.00000000e+00],
       [-2.16690351e+03,  0.00000000e+00,  4.62815785e-02, ...,
        -3.51230899e+02, -4.56382646e+02,  4.59576075e+01],
       [-2.11777561e+03,  0.00000000e+00,  4.62815785e-02, ...,
        -3.39891913e+02, -4.27294674e+02,  1.31231416e+02],
       ...,
       [-3.58682233e+03, -2.20300314e+04,  0.00000000e+00, ...,
         4.62815786e+01,  2.31407893e+01, -3.09309046e+06],
       [-3.58682233e+03, -2.20300314e+04,  0.00000000e+00, ...,
         4.62815786e+01,  2.31407893e+01, -3.10919644e+06],
       [-3.58682233e+03, -2.20068906e+04,  0.00000000e+00, ...,
         6.94223677e+01,  4.62815785e+01, -3.15001680e+06]])